In [9]:
###TODO:
# cal comparison does not account for pages i think.


#imports
from tkinter import *
import os
import re
import time
import datetime

import telnetlib
import getpass 
import sys
import time
import serial
import threading #potentially replaced by multiprocessing
from multiprocessing.pool import ThreadPool

import pandas as pd
import numpy as np

from pdfrw import PdfReader
import matplotlib.pyplot as plt
import PyPDF2
from tkinter.filedialog import askopenfilename
os.environ.setdefault('MAGICK_HOME', '/usr/local/Cellar/imagemagick@6/6.9.9-34/')
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io


#global defines
nport_ip = "10.136.1.99"
ref_port = "4001"



In [8]:
#generate "main" gui window in tkinter
def onclick(): 
        pass

#updates the main status label with given text.  Could be accomplished by just calling "main_status.config" from within the main function...
def update_txt(next_instruction):
    main_status.config(text=next_instruction)   
    
#def connex(port):
    #print(port)
    
win = Tk()

win.title('SBE CTD Prequalification')
win.geometry('800x600') # Size 200, 200

text = Text(win, height=2)

#labels
main_status = Label(win, text="Waiting for device selection")
calsheet_status_label = Label(win, text="")
sample_status_label = Label(win, text="")

port2 = Button(win, text="4002 - 485", command=lambda: instrument_connection("4002"))
port3 = Button(win, text="4003 - 232", command=lambda: instrument_connection("4003"))
port4 = Button(win, text="4004", command=lambda: instrument_connection("4004"))

text.insert(INSERT, "Please connect the RMA device and specify the port used")

text.pack()
port2.pack()
port3.pack()
port4.pack()
main_status.pack()
calsheet_status_label.pack()
sample_status_label.pack()

#btn = Button(win, text='Hello', command=hello)
#btn.pack(expand=YES, fill=BOTH)

mainloop()


bricked it


Exception in Tkinter callback
Traceback (most recent call last):
  File "<ipython-input-3-9a667576b4d9>", line 17, in instrument_connection
    ref_telnet = telnetlib.Telnet(nport_ip, ref_port) #open connection
  File "/Users/mtceleccoop1/anaconda3/lib/python3.6/telnetlib.py", line 218, in __init__
    self.open(host, port, timeout)
  File "/Users/mtceleccoop1/anaconda3/lib/python3.6/telnetlib.py", line 234, in open
    self.sock = socket.create_connection((host, port), timeout)
  File "/Users/mtceleccoop1/anaconda3/lib/python3.6/socket.py", line 724, in create_connection
    raise err
  File "/Users/mtceleccoop1/anaconda3/lib/python3.6/socket.py", line 713, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mtceleccoop1/anaconda3/lib/python3.6/tkinter/__init__.py", line 1699, in __call__
    return self.func(*args)
  File 

In [3]:
# Instrument Connection
#module to:
# 1. Prompt user to select NPORT port RMA instrument is connected to (maybe goes in main gui window?)
# 2. Create a telnet connection to both instruments
# 3. Send wakeup to both instruments
# 4. Alert user that connection successful

#entry point to module
def instrument_connection(rma_port):
    global nport_ip
    global ref_port
    device_dict = {
        "4002" : "id?\r\n",
        "4003" : "\r\n"
    }
    try:
        ref_telnet = telnetlib.Telnet(nport_ip, ref_port) #open connection
        rma_telnet = telnetlib.Telnet(nport_ip, rma_port)
        update_txt("Connecting, please wait...")
        time.sleep(1)
        ref_telnet.write(device_dict['4003'].encode("ascii"))  #wakeup both instruments
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_telnet.write((device_dict[rma_port]).encode("ascii"))
        time.sleep(1)
        rma_ok = rma_telnet.read_very_eager()
        ref_ok = ref_telnet.read_very_eager()
        #print(str(ref_ok))
        #print(str(rma_ok))
        ref_telnet.close() #close reference connection, not done with rma yet
        rma_telnet.close()
    except:
        print("bricked it")
        ref_telnet.close()
        rma_telnet.close()
    finally:
        ref_telnet.close()
        rma_telnet.close()
    update_txt("Connection Successful")
    calsheet_status_label.config(text="Collecting RMA calibration data...")    
    compcals = compare_cals_entrypoint(rma_port) #pass the telnet connection to compare_cals function    
    print(compcals)
    

In [4]:
# Compare Samples
#module to:
# 1. Collect 10 samples (1/min) from each instrument
# 2. Compare samples at like timestamps(should be +/- 1s)
# 3. Format data and present to user, advise pass/fail, generate comparison column as well

def connect_ref(s):
    tn = s
    tn.write(("\r\n").encode('ascii'))
    time.sleep(2)
    sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S:%f')
    tn.write(("TS\r\n").encode('ascii'))
    time.sleep(60)
    data = str(tn.read_very_eager())
    data += sample_time
    print(data)
    tn.close()

def connect_rma(s):
    tn = s
    tn.write(("id?\r\n").encode('ascii'))
    time.sleep(1)
    sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S:%f')
    tn.write(("#03TS\r\n").encode('ascii'))
    time.sleep(60)
    data = str(tn.read_very_eager())
    data += sample_time
    tn.close()

    
def collect_samples():
    global nport_ip
    rma_tn = telnetlib.Telnet(nport_ip, "4002", 5)
    ref_tn = telnetlib.Telnet(nport_ip, "4001", 5)    
    procRMA = pool.apply_async(connect_rma, (rma_tn,))
    procREF = pool.apply_async(connect_ref, (ref_tn,))
    rma_sample = procRMA.get()
    ref_sample = profREF.get()
    
#     threadRMA = threading.Thread(target=connect_rma, args=(rma_tn,),).start()
#     threadREF = threading.Thread(target=connect_ref, args=(ref_tn,),).start()
    
    

In [5]:
# Compare Cals
#module to:
# 1. Prompt user for calibration sheet location
# 2. Collect cals from instrument(dcal, #iidc)
# 3. Compare Results
# 4. Display side by side results AND suggest an action(pass/fail)
# 5. If "pass" then continue after ~30seconds if "fail" pause and wait for user?\
def compare_cals_entrypoint(rma_port):
    if rma_port == "4002":
        ins_caldata = get_cals_37()
    if rma_port == "4003":
        ins_caldata = get_cals_19()
    sheet_caldata = get_sheet_cals()
    df = pd.DataFrame(data = compare_cals(ins_caldata, sheet_caldata), columns=['Instrument', 'Sheet'])
    return df
    
def get_cals_37():
    global nport_ip
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    sbe_id = str(rma_telnet.read_very_eager())[5:7]
    rma_telnet.write(("#"+sbe_id+"DC\r\n").encode("ascii"))
    time.sleep(1)
    #print(rma_telnet.read_very_eager()) #cal data to be parsed
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return str(caldata)
def get_cals_19():
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("DCal\r\n".encode("ascii"))
    time.sleep(1)
    #print(rma_telnet.read_very_eager()) #cal data to be parsed
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return str(caldata)

#this function doesn't actually need to exist, but helps with clarity i think?
def get_sheet_cals():
    filepath = request_filepath()
    return read_file(filepath)
    

#get user input filepath(browse files)    
def request_filepath():
    file_select = Tk()
    file_select.withdraw()
    filename=askopenfilename()
    file_select.update()
    file_select.destroy()
    return filename

#OCR on file, parse out to text
def read_file(filepath):
    tool = pyocr.get_available_tools()[0]
    lang = tool.get_available_languages()[0]
    req_image = []
    final_text = []
    image_pdf = Image(filename=filepath, resolution=300)
    image_jpeg = image_pdf.convert('jpeg')
    for img in image_jpeg.sequence:
        img_page = Image(image=img)
        req_image.append(img_page.make_blob('jpeg'))
    for img in req_image: 
        txt = tool.image_to_string(
            PI.open(io.BytesIO(img)),
            lang=lang,
            builder=pyocr.builders.TextBuilder()
        )
        final_text.append(txt)
    merge_text = ''.join(final_text)
    return merge_text
    
#compare cal file and instruments put into 2 column dataframe 
#no comparison yet just collects and columnizes

#pass strings for cal sheet and instrument cals and it will compare all values and generate a comparison report
def compare_cals(instr, sheet):
    i_s = instr
    s_s = sheet
    temp_tags = [r"(?<![pP])([aA]0 = )[^\\\n]*", r"(?<![pP])([aA]1 = )[^\\\n]*", r"(?<![pP])([aA]2 = )[^\\\n]*", r"(?<![pP])([aA]3 = )[^\\\n]*"]#"([aA]1 = )[^\\\]*", "([aA]2 = )[^\\\]*", "([aA]3 = )[^\\\]*"]
    press_tags = [r"(?i)(pa0 = )[^\\\s]*", r"(?i)(pa1 = )[^\\\s]*", r"(?i)(pa2|paz)( = )[^\\\s]*", 
                  r"(?i)(ptca0 = )[^\\\s]*", r"(?i)(ptca1 = )[^\\\s]*", r"(?i)(ptca2|ptcaz)( = )[^\\\s]*",
                  r"(?i)(ptempa0)[^\\\s]*", r"(?i)(ptempa1)[^\\\s]*", r"(?i)(ptempa2|ptempaz)( = )[^\\\s]*",
                  r"(?i)(ptcb0 = )[^\\\s]*", r"(?i)(ptcb1 = )[^\\\s]*", r"(?i)(ptcb2|ptcbz)( = )[^\\\s]*"] 
    cond_tags = [r"(?i)(g = )[^\\\s]*", r"(?i)(h = )[^\\\s]*", r"(?i)(i = )[^\\\s]*", r"(?i)(j = )[^\\\s]*", r"(?i)(CPcor = )[^\\\s]*", r"(?i)(CTcor = )[^\\\s]*"]
    comparison_array = []
    for tag in temp_tags:
        instr_press = find_value(tag, i_s)
        cal_press = find_value(tag, s_s)
        comparison_array.append((instr_press, cal_press))
    for tag in cond_tags:
        instr_press = find_value(tag, i_s)
        cal_press = find_value(tag, s_s)
        comparison_array.append((instr_press, cal_press))
    for tag in press_tags:
        instr_press = find_value(tag, i_s)
        cal_press = find_value(tag, s_s)
        comparison_array.append((instr_press, cal_press))    
    return comparison_array
    
def find_value(tag, input_file):
    m = re.search(tag, input_file)
    if m:
        return m.group(0)
    else:
        return "Not Found"


In [ ]:
# Cleanup
# 1. Close telnet connections
# 2. Save any relevant tables/data to files(not sure if any exists)



In [5]:
#noodle tab
def test_func():
    global nport_ip
    test = nport_ip
    return test

print(test_func())

10.136.1.99


In [89]:
try:
    two = 1+1
except:
    three = 1+1
    
print(two)

2
